In [1]:
import numpy as np
import os
import pdb
import matplotlib.pyplot as plt

from numpy.linalg import eig, eigh
import math

fd = open('train-images-idx3-ubyte')
loaded = np.fromfile(file=fd, dtype=np.uint8)
trData = loaded[16:].reshape((60000, 28*28)).astype(float)

PermissionError: [Errno 13] Permission denied: 'train-images-idx3-ubyte'

In [47]:
def one_hot(x, n):
    if type(x) == list:
        x = np.array(x)
    x = x.flatten()
    o_h = np.zeros((len(x), n))
    o_h[np.arange(len(x)), x] = 1
    return o_h

In [50]:
def mnist(noTrSamples=1000, noTsSamples=100, digit_range=[0, 10], noTrPerClass=100, noTsPerClass=10):
    #data_dir = os.path.join(datasets_dir, 'PCA_1215126783/')
    fd = open('train-images-idx3-ubyte')
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trData = loaded[16:].reshape((60000, 28*28)).astype(float)

    fd = open(os.path.join(data_dir, 'train-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trLabels = loaded[8:].reshape((60000)).astype(float)

    fd = open(os.path.join(data_dir, 't10k-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    tsData = loaded[16:].reshape((10000, 28*28)).astype(float)

    fd = open(os.path.join(data_dir, 't10k-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    tsLabels = loaded[8:].reshape((10000)).astype(float)

    trData = trData/255.
    tsData = tsData/255.

    tsX = np.zeros((noTsSamples, 28*28))
    trX = np.zeros((noTrSamples, 28*28))
    tsY = np.zeros(noTsSamples)
    trY = np.zeros(noTrSamples)

    count = 0
    for ll in range(digit_range[0], digit_range[1]):
        # Train data
        idl = np.where(trLabels == ll)
        idl = idl[0][: noTrPerClass]
        idx = list(range(count*noTrPerClass, (count+1)*noTrPerClass))
        trX[idx, :] = trData[idl, :]
        trY[idx] = trLabels[idl]
        # Test data
        idl = np.where(tsLabels == ll)
        idl = idl[0][: noTsPerClass]
        idx = list(range(count*noTsPerClass, (count+1)*noTsPerClass))
        tsX[idx, :] = tsData[idl, :]
        tsY[idx] = tsLabels[idl]
        count += 1
    
    np.random.seed(1)
    test_idx = np.random.permutation(tsX.shape[0])
    tsX = tsX[test_idx,:]
    tsY = tsY[test_idx]

    trX = trX.T
    tsX = tsX.T
    trY = trY.reshape(1, -1)
    tsY = tsY.reshape(1, -1)
    return trX, trY, tsX, tsY

In [51]:
trX, trY, tsX, tsY = mnist(noTrSamples=1000, noTsSamples=100, digit_range=[0, 10], noTrPerClass=100, noTsPerClass=10)

FileNotFoundError: [Errno 2] No such file or directory: 'PCA_1215126783/train-images-idx3-ubyte'

In [10]:
############################################################################################################################
                                    # QUESTION 1 #
############################################################################################################################

cov_matrix = np.cov(trX);
    
eigen_values, eigen_vectors = eigh(cov_matrix)

# Taking just first 100 Eigen Vectors and eigen values
E = eigen_vectors.T[::-1][:100,:].T
E_lambda = np.diag(eigen_values[::-1][:100])

# Applying PCA to reduce the dataset
reduced_trX = np.dot(E.T, trX)
    
# Percentage of variance captured
variance = np.trace(E_lambda) / np.sum(eigen_values)
print ("Variance = "+str(variance*100))

# Obtain the new co-variance matrix and plotting
reduced_cov_matrix = np.cov(reduced_trX)
plt.matshow(reduced_cov_matrix)
plt.title("Covariance Matrix of Reduced Training Set after PCA");

print ("Analysis: The covariance is observed to be kind of a Diagonal matrix where the variances are seen to decrease. This is because PCA projects the points on the directions of maximum variance. These directions are the eigen vectors. These eigen vectors are arranged based on decreasing order of eigen value, which is proportional to variance. This explains the decreasing variance from high to low in the diagonal. The other covariances are 0 due to the property of PCA to choose eigen vectors to preserve variance.")

    
# Plotting the images for all numbers
for digit in range(10):
    plt.figure()
    plt.imshow(np.reshape(reduced_trX[:, digit*100], (10, 10)))
    plt.title("Digit After PCA "+str(digit))
plt.show()
    

NameError: name 'trX' is not defined

In [11]:
############################################################################################################################
                                    # QUESTION 2 #
############################################################################################################################

# Question 2
# Find Eigen values and Eigen vectors for reduced dimensions
E_lambda_reduced, E_reduced = eigh(reduced_cov_matrix)

# Eliminate all negative values
#E_lambda_reduced[E_lambda_reduced < 0] = 0
    
# Change the eigenvalue matrix for whitening
E_lambda_reduced = np.diag(1 / np.sqrt(E_lambda_reduced[::-1] + math.exp(-3)))
    
# FInd Ew
Ew = np.dot(E_reduced.T[::-1].T, E_lambda_reduced)

# Whitened data
reduced_trX_PCA_whitened = np.dot(Ew.T, reduced_trX)
    
# Find the new covariance matrix for PCA Whitening and plot
cov_PCA_whitened = np.cov(reduced_trX_PCA_whitened)
plt.matshow(cov_PCA_whitened)
plt.title("Covariance matrix for reduced data after PCA whitening")

# Applying ZCA transform
reduced_trX_ZCA_whitened = np.dot(E_reduced, reduced_trX_PCA_whitened)

# Find new covariance matrix for ZCA whitening and plot
cov_ZCA_whitening = np.cov(reduced_trX_ZCA_whitened)
plt.matshow(cov_ZCA_whitening)
plt.title("Covariance matrix for reduced data after ZCA whitening")

print ("Analysis: The covariance matrices After PCA and ZCA Whitening are very close to Identity matrices which is the desired result after whitening ")


# Plot images for digits after ZCA
for digit in range(10):
    plt.figure()
    plt.imshow(np.reshape(reduced_trX_ZCA_whitened[:, digit*100 + 25], (10, 10)))
    plt.title("Digit after ZCA "+str(digit))
plt.show()


NameError: name 'reduced_cov_matrix' is not defined

In [12]:
############################################################################################################################
                                    # QUESTION 3 #
############################################################################################################################

trX_class0, trX_class1 = reduced_trX[:, :100], reduced_trX[:, 100:200]

# Find Sw matrix
Sw = np.cov(trX_class0) + np.cov(trX_class1)

# Find inverse of Sw matrix
Sw_inverse = np.linalg.inv(Sw)

# Find the two means
m0, m1 = np.mean(trX_class0, axis=1), np.mean(trX_class1, axis=1)
mean_diff = m0 - m1

# Find the fisher linear discriminant
W = np.dot(Sw_inverse, mean_diff.reshape(100,1))

# Find the threshold
threshold_value = 0.5 * np.dot(W.T, m0 + m1)
print ("Threshold value = "+str(threshold_value[0]))
print ("LDA Result : If y > "+str(threshold_value[0])+" then predict 0, else predict 1.")

# Plot the Results of LDA on the Training set
reduced_trX_LDA = np.dot(W.T, reduced_trX[:, :200])
plt.figure()
plt.plot(reduced_trX_LDA[:, :100], np.zeros_like(reduced_trX_LDA[:, :100]), 'gx', label = "Digit 0")
plt.plot(reduced_trX_LDA[:, 100:200], np.zeros_like(reduced_trX_LDA[:, 100:200]), 'ro', label = "Digit 1")
plt.title("LDA results on the training set")

# Find eigenvalues and eigen vectors of test set
E_lambda_test, E_test = eigh(np.cov(tsX))

# Find reduced Test set
reduced_tsX = np.dot(E_test.T[::-1][:100, :], tsX)

# Find the test set for classes 0 and 1
required_ordered_indices_labels = np.argsort(tsY[0])[:20]

# Project the test points on W
reduced_tsX_LDA = np.dot(W.T, reduced_tsX[:, required_ordered_indices_labels])

plt.figure()
plt.plot(reduced_tsX_LDA[:, :10], np.zeros_like(reduced_tsX_LDA[:, :10]), 'gx')
plt.plot(reduced_tsX_LDA[:, 10:20], np.zeros_like(reduced_tsX_LDA[:, 10:20]) + 0, 'ro')
plt.title("LDA results on the test set")

# Find the predictions and compute accuracy
true_predictions = tsY[0][required_ordered_indices_labels]
error = 0

for i, prediction in enumerate(reduced_tsX_LDA[0]):
    if prediction <= threshold_value:
        prediction = 1
    else:
        prediction = 0
        
    if prediction != true_predictions[i]:
        error += 5

print ("Error in classifying Test set in LDA = "+str(error)+"%")

plt.show()

NameError: name 'reduced_trX' is not defined